# Build a simple LLM Application with LCEL

In this quickstart we'll show you how to build a simple LLM application with Langchain. This application will translate text from English into another language. This is a relatively simple LLM spplication - it's just a single LLM call pluss some prompting. Still, this is a great way to get started with Langchain - a lot of features can be built with just some prompting and an LLM call !

After seeing this video, you'll have a high level overview of:
- Using langchain models
- Using PromptTempleate and OutputParsers
- Using Langchain Expression Language (LCEL) to chain components together
- Debugging and tracing your application using LangSmith
- Deploying your application with LangServe

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACKING_V2'] = "True"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It",groq_api_key = groq_api_key)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f19a0874140>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f19a0874bf0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content = "Translate the following from English to French"),
    HumanMessage(content="Hello How are you")
]

result = model.invoke(messages)
print(result)

content='Bonjour, comment allez-vous ? \n' response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 25, 'total_tokens': 36, 'completion_time': 0.022, 'prompt_time': 0.002626489, 'queue_time': None, 'total_time': 0.024626488999999998}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-26a1377f-f80d-4884-8242-19eb1391a866-0' usage_metadata={'input_tokens': 25, 'output_tokens': 11, 'total_tokens': 36}


In [8]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

In [9]:
## Using LCEL - chain the componenets

chain = model | parser

chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [10]:
## Prompt Templates 

from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {langauage}"

prompt = ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)


In [14]:
result = prompt.invoke({"langauage":"French","text":"Hello"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French'), HumanMessage(content='Hello')])

In [17]:
chain = prompt | model | parser

chain.invoke({"langauage":"French","text":"Hello"})

'Bonjour! \n'